In [2]:
!pip install gradio -q

In [3]:
!pip install torch torchvision huggingface_hub

In [22]:
!pip install gradio huggingface_hub

In [23]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import json
import gradio as gr
from torchvision.models import resnet50
from huggingface_hub import Repository, login

In [12]:
from model import resnet50

In [13]:
#model = resnet50()
model_path = "./Checkpoint/checkpoint-epcoh24_lr_point1"

In [14]:
def load_model(model_path):
    model = resnet50()  # Create an instance of the ResNet-50 model
    state_dict = torch.load(model_path, map_location='cpu')  # Load the state dictionary

    # # Remove "module." prefix if present
    # if 'module.' in next(iter(state_dict.keys())):
    #     state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}

    model.load_state_dict(state_dict)  # Load the state dictionary
    model.eval()  # Set the model to evaluation mode
    return model

In [22]:
# from collections import OrderedDict

# # Load the checkpoint
# checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))

# # If the checkpoint has a "state_dict" key, use it
# state_dict = checkpoint["state_dict"] if "state_dict" in checkpoint else checkpoint

# # Filter or rename keys to match the ResNet-50 architecture
# new_state_dict = OrderedDict()
# for key, value in state_dict.items():
#     if key.startswith("module."):  # Remove 'module.' if checkpoint was trained with DataParallel
#         new_key = key[7:]
#     else:
#         new_key = key
#     if new_key in model.state_dict():  # Only add keys that exist in the ResNet-50 architecture
#         # Check for shape mismatch and resize if necessary
#         if value.shape != model.state_dict()[new_key].shape:
#             print(f"Resizing {new_key} from {value.shape} to {model.state_dict()[new_key].shape}")
#             value = value.reshape(model.state_dict()[new_key].shape)  # Resize the tensor

#         new_state_dict[new_key] = value
#     # else:
#     #     print(f"Skipping key: {new_key} not found in model")  # Print skipped keys for debugging

# # Load the filtered state_dict into the model
# model.load_state_dict(new_state_dict, strict=False)

<ipython-input-22-142844de7ef9>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))


_IncompatibleKeys(missing_keys=['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.conv3.weight', 'layer1.1.bn3.weight', 'layer1.1.bn3.bias', 'layer1.1.bn3.runni

In [23]:
# # save the above nmodel with new name
# model_save_dir = "/content/model_colab"
# torch.save(model.state_dict(), model_save_dir)

In [ ]:
# checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))
# print("Checkpoint keys:", checkpoint.keys())

# if "state_dict" in checkpoint:
#     print("State dict keys:", checkpoint["state_dict"].keys())
# else:
#     print("State dict keys:", checkpoint.keys())


Checkpoint keys: dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'loss'])
State dict keys: dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'loss'])


<ipython-input-5-6bf15358f4bd>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))


In [ ]:
# tokenizer_path = "/content/imagenet_class_index.json"

In [15]:
def preprocess_image(image):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    image = image.convert("RGB")  # Ensure image is in RGB format
    image = preprocess(image)  # Apply transformations
    image = image.unsqueeze(0)  # Add batch dimension
    return image

In [16]:
# Prediction function
def predict(image):
    image_tensor = preprocess_image(image)
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        top5_probabilities, top5_indices = probabilities.topk(5)

    results = {}
    for i in range(5):
        class_index = top5_indices[0][i].item()
        class_label = class_labels.get(str(class_index), "Unknown class")
        results[class_label] = top5_probabilities[0][i].item()  # Store label and probability in a dictionary
    print("See the prediction result : ", results)
    return results  # Return the results as a dictionary

In [13]:
#tokenizer_path = "/content/imagenet_class_index.json"

In [17]:
# Load class index mapping
def load_class_labels(label_path):
    with open(label_path, 'r') as f:
        class_labels = json.load(f)
    return class_labels

In [18]:
def load_model(model_path):
    model = resnet50()  # Create an instance of the ResNet-50 model
    checkpoint = torch.load(model_path, map_location='cpu')  # Load the checkpoint

    # Access the 'model_state_dict' key within the checkpoint
    state_dict = checkpoint['model_state_dict']

    # Remove "module." prefix if present (common when using DataParallel)
    if 'module.' in next(iter(state_dict.keys())):
        state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}

    model.load_state_dict(state_dict)  # Load the state dictionary
    model.eval()  # Set the model to evaluation mode
    return model

In [19]:
# Load model and class labels
# model_path = 'model.pt'  # Path to the trained model
model_path = "./Checkpoint/checkpoint-epcoh24_lr_point1"
label_path = 'imagenet_class_index.json'  # Path to the class index mapping
model = load_model(model_path)
class_labels = load_class_labels(label_path)

In [20]:
# Create Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=5),
    title="Image Classification using ResNet 50 Model trained on Imagenet 1000 dataset",
    description="Upload an image to get the top-5 predictions."
)



In [21]:
# Launch the app
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


See the prediction result :  {'lion, king of beasts, Panthera leo': 0.9751031994819641, 'Tibetan mastiff': 0.017351044341921806, 'chow, chow chow': 0.003145449096336961, 'Persian cat': 0.0004533762112259865, 'Pekinese, Pekingese, Peke': 0.0002895498473662883}
See the prediction result :  {'German shepherd, German shepherd dog, German police dog, alsatian': 0.9453887343406677, 'malinois': 0.02955769933760166, 'kelpie': 0.007483848370611668, 'dingo, warrigal, warragal, Canis dingo': 0.0026456655468791723, 'Australian terrier': 0.0022577173076570034}
See the prediction result :  {'German shepherd, German shepherd dog, German police dog, alsatian': 0.9453887343406677, 'malinois': 0.02955769933760166, 'kelpie': 0.007483848370611668, 'dingo, warrigal, warragal, Canis dingo': 0.0026456655468791723, 'Australian terrier': 0.0022577173076570034}
See the prediction result :  {'tree frog, tree-frog': 0.8318064212799072, 'tailed frog, bell toad, ribbed toad, tailed toad, Ascaphus trui': 0.149111643